# Compare LiDAR odometry methods

Adjusting factors such as: covariance values, downsample rate, Lidar/GPS relative weighting

In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time

import symforce
try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass 

import symforce.symbolic as sf

import chimera_fgo.util.general as general
from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results

%load_ext autoreload
%autoreload 2

Load data

In [ ]:
kitti_seq = '0027'
start_idx = 1550 if kitti_seq == '0028' else 0

gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

N = len(gt_enu)
N_PLOT = 2000

Odometry positions from ICP results

In [ ]:
# Plot against ground-truth
fig = go.Figure()
fig.add_trace(go.Scatter(x=gt_enu[:N_PLOT, 0], y=gt_enu[:N_PLOT, 1], mode='lines', name='Ground truth'))
fig.add_trace(go.Scatter(x=lidar_positions[:N_PLOT, 0], y=lidar_positions[:N_PLOT, 1], mode='lines', name='Lidar odometry'))
fig.update_layout(title='Lidar odometry vs ground truth', width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

Pose composition

In [ ]:
new_lidar_positions = np.zeros_like(lidar_positions)

R_abs = gt_Rs[0].copy()
t_abs = gt_enu[0].copy()
new_lidar_positions[0] = t_abs

start_time = time.time()
for i in range(1, N):
    t_abs += (R_abs @ lidar_ts[i-1]).flatten()
    #t_abs += R_abs.dot(lidar_ts[i-1])
    R_abs = lidar_Rs[i-1] @ R_abs
    #R_abs = lidar_Rs[i-1].dot(R_abs)
    new_lidar_positions[i] = t_abs
print(f"Time taken: {time.time() - start_time}")

In [ ]:
new_lidar_positions

In [ ]:
# Lidar odometry
lidar_odom = [None] * (N - 1)
for i in range(N - 1):
    lidar_odom[i] = (lidar_Rs[i], lidar_ts[i])

In [ ]:
init_poses = [sf.Pose3.identity()]
init_pose = (init_poses[0].R.to_rotation_matrix().to_numpy(), init_poses[0].t.to_numpy().flatten())

In [ ]:
from chimera_fgo.odometry import odometry

#init_pose = (gt_Rs[0].copy(), gt_enu[0].copy())
positions, _ = odometry(init_pose, lidar_odom)

In [ ]:
positions

Symforce pose composition

In [ ]:
sym_lidar_positions = np.zeros_like(lidar_positions)

R_abs = gt_Rs[0].copy()
t_abs = gt_enu[0].copy()
T_abs = sf.Pose3(sf.Rot3.from_rotation_matrix(R_abs), sf.V3(t_abs))
sym_lidar_positions[0] = t_abs

start_time = time.time()
for i in range(1, N):
    T_icp = sf.Pose3(sf.Rot3.from_rotation_matrix(lidar_Rs[i-1]), sf.V3(lidar_ts[i-1]))
    T_abs = T_abs * T_icp
    sym_lidar_positions[i] = T_abs.t
print(f"Time taken: {time.time() - start_time}")

In [ ]:
# Plot against ground-truth
fig = go.Figure()
fig.add_trace(go.Scatter(x=gt_enu[:N_PLOT, 0], y=gt_enu[:N_PLOT, 1], mode='lines', name='Ground truth'))
fig.add_trace(go.Scatter(x=sym_lidar_positions[:N_PLOT, 0], y=sym_lidar_positions[:N_PLOT, 1], mode='lines', name='Lidar odometry'))
fig.update_layout(title='Lidar odometry vs ground truth', width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

Sliding window FGO (lidar only)

In [ ]:
from chimera_fgo.symforce.fgo_lidar_only import fgo_lidar_only



Sliding window FGO (scaled GPS sigma)